In [1]:
#############################################################
# 1. Libraries

import os
import datetime
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
from PIL import Image

import matplotlib
import matplotlib.pyplot as plt
pd.options.display.max_columns = None    # disp all columns

import tensorflow as tf
tf.keras.backend.clear_session()

physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')
    
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers, models, optimizers, regularizers, losses, metrics
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

import albumentations
import gc

#############################################################

In [2]:
#############################################################
# 2. Paths & Global Variables

## 2.1 Paths

path = '../../01_Data/'
path_sequences = path + '01_GeneratedSequences/'
path_spectograms = path + '04_GeneratedSpectogramsSTFT/'


df_train = pd.read_csv(path + 'train.csv')
df_sample_submission = pd.read_csv(path + 'sample_submission.csv') 

## 2.2 Global Variables

unique_segments_id_train = set(df_train['segment_id'])
unique_segments_id_test = set(df_sample_submission['segment_id'])

SEQ_LENGTH = 60_001
IMG_SIZE = (128, 469)

#############################################################

In [3]:
#############################################################
# 3. Preprocess

dict_segments_paths_train = {
    segment : path + 'train/' + str(segment) + '.csv' for segment in unique_segments_id_train
}

dict_segments_paths_test = {
    segment : path + 'test/' + str(segment) + '.csv' for segment in unique_segments_id_test
}

###

dict_segments_sequences_paths_train = {
    segment : path_sequences + 'train/' + str(segment) + '.npy' for segment in unique_segments_id_train
}

dict_segments_sequences_paths_test = {
    segment : path_sequences + 'test/' + str(segment) + '.npy' for segment in unique_segments_id_test
}

###

dict_segments_spectograms_paths_train = {
    segment : path_spectograms + 'train/' + str(segment) + '/'
    for segment in unique_segments_id_train
}

dict_segments_spectograms_paths_test = {
    segment : path_spectograms + 'test/' + str(segment) + '/'
    for segment in unique_segments_id_test
}

###

df_train['time_to_eruption'] = df_train['time_to_eruption']/(10**6)

dict_labels = {
    segment : df_train['time_to_eruption'][df_train['segment_id']==segment].values.flatten()
    
    for segment in unique_segments_id_train
}


#############################################################

In [6]:
#############################################################
# 5. Data Generator

def displayImage(image, figsize=(20, 15)):
    fig = plt.figure(figsize=figsize)
    plt.grid(False)
    plt.imshow(image)
    plt.show()


# 5.1 - Data Augmentation

def getTrainTransforms():
    return albumentations.Compose([
            albumentations.OneOf([
                albumentations.GaussNoise(p=0.2),
                albumentations.Cutout(num_holes=8, max_h_size=12, max_w_size=12, fill_value=0, p=0.2),
            ], p=0.3),
        albumentations.OpticalDistortion(p=0.3),
        albumentations.ShiftScaleRotate(shift_limit=0.05, rotate_limit=1, p=0.5),
        albumentations.RandomCrop(IMG_SIZE[0]-10, IMG_SIZE[1]-10, p=0.5),
        albumentations.PadIfNeeded(min_height=IMG_SIZE[0], min_width=IMG_SIZE[1], value=0, p=1.0)
    ])

class VolcanoSequencesGenerator(Sequence):
    
    def __init__(self, segments, path_spectograms, batch_size, dict_labels, transforms, training=True):
        super(VolcanoSequencesGenerator, self).__init__()
        
        self.segments = segments
        self.path_spectograms = path_spectograms
        self.batch_size = batch_size
        self.dict_labels = dict_labels
        self.transforms = transforms
        self.training = training
        self.on_epoch_end()
        
    def __len__(self):
        self.num_steps = int(np.ceil(len(self.segments) / self.batch_size))
        return self.num_steps
        
    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        list_batch_segments = [self.segments[k] for k in indexes]
        
        array_spectograms_s0 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_0.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s1 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_1.npy', 
                                                            allow_pickle=True).transpose(1, 0) 
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s2 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_2.npy', 
                                                            allow_pickle=True).transpose(1, 0) 
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s3 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_3.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s4 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_4.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s5 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_5.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s6 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_6.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s7 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_7.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s8 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_8.npy', 
                                                            allow_pickle=True).transpose(1, 0) 
                                     for segment in list_batch_segments]).astype(np.uint8) 
        array_spectograms_s9 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_9.npy', 
                                                            allow_pickle=True).transpose(1, 0) 
                                     for segment in list_batch_segments]).astype(np.uint8)  
        
        if self.transforms:
            data_s0, data_s1 = {'image':array_spectograms_s0}, {'image':array_spectograms_s1}
            data_s2, data_s3 = {'image':array_spectograms_s2}, {'image':array_spectograms_s3}
            data_s4, data_s5 = {'image':array_spectograms_s4}, {'image':array_spectograms_s5}
            data_s6, data_s7 = {'image':array_spectograms_s6}, {'image':array_spectograms_s7}
            data_s8, data_s9 = {'image':array_spectograms_s8}, {'image':array_spectograms_s9}
            
            array_spectograms_s0 = np.stack([self.transforms(image=x)['image'] for x in data_s0['image']], axis=0)
            array_spectograms_s1 = np.stack([self.transforms(image=x)['image'] for x in data_s1['image']], axis=0)
            array_spectograms_s2 = np.stack([self.transforms(image=x)['image'] for x in data_s2['image']], axis=0)
            array_spectograms_s3 = np.stack([self.transforms(image=x)['image'] for x in data_s3['image']], axis=0)
            array_spectograms_s4 = np.stack([self.transforms(image=x)['image'] for x in data_s4['image']], axis=0)
            array_spectograms_s5 = np.stack([self.transforms(image=x)['image'] for x in data_s5['image']], axis=0)
            array_spectograms_s6 = np.stack([self.transforms(image=x)['image'] for x in data_s6['image']], axis=0)
            array_spectograms_s7 = np.stack([self.transforms(image=x)['image'] for x in data_s7['image']], axis=0)
            array_spectograms_s8 = np.stack([self.transforms(image=x)['image'] for x in data_s8['image']], axis=0)
            array_spectograms_s9 = np.stack([self.transforms(image=x)['image'] for x in data_s9['image']], axis=0)
                 
        batch = (array_spectograms_s0/255, array_spectograms_s1/255, array_spectograms_s2/255, array_spectograms_s3/255, 
                 array_spectograms_s4/255, array_spectograms_s5/255, array_spectograms_s6/255, array_spectograms_s7/255, 
                 array_spectograms_s8/255, array_spectograms_s9/255)       
            
        if self.training:
            array_labels = np.asarray([self.dict_labels[segment] for segment in list_batch_segments])
            return batch, array_labels
        else:
            return batch
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.segments))
        np.random.shuffle(self.indexes)
        
        
    def generateOrderedSequences(self, list_segments):
        array_spectograms_s0 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_0.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s1 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_1.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s2 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_2.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s3 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_3.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s4 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_4.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s5 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_5.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s6 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_6.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s7 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_7.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s8 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_8.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
        array_spectograms_s9 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_9.npy', 
                                                            allow_pickle=True).transpose(1, 0)
                                     for segment in list_segments]).astype(np.uint8) 
                 
        batch = (array_spectograms_s0/255, array_spectograms_s1/255, array_spectograms_s2/255, array_spectograms_s3/255, 
                 array_spectograms_s4/255, array_spectograms_s5/255, array_spectograms_s6/255, array_spectograms_s7/255, 
                 array_spectograms_s8/255, array_spectograms_s9/255)        
        
        if self.training:
            array_labels = np.asarray([self.dict_labels[segment] for segment in list_segments])
        if self.training:
            return batch, array_labels
        else:
            return batch
    
#############################################################

In [5]:
# g = VolcanoSequencesGenerator(list(unique_segments_id_train), dict_segments_spectograms_paths_train,
#                                                   batch_size=2, dict_labels=dict_labels,
#                                                   transforms=getTrainTransforms(),
#                                                   training=True)

# for batch in g:
#     break

# print(batch[0][1].shape)
# print(batch[0][0].shape, batch[0][1].shape, batch[0][2].shape, batch[0][3].shape, batch[0][4].shape, batch[0][5].shape,
#       batch[0][6].shape, batch[0][7].shape, batch[0][8].shape, batch[0][9].shape)

# displayImage(batch[0][0][0], figsize=(12, 10))

In [6]:
#############################################################
# 6. Models

class ReturnBestEarlyStopping(tf.keras.callbacks.EarlyStopping):
    def __init__(self, **kwargs):
        super(ReturnBestEarlyStopping, self).__init__(**kwargs)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            if self.verbose > 0:
                print(f'\nEpoch {self.stopped_epoch + 1}: early stopping')
        elif self.restore_best_weights:
            if self.verbose > 0:
                print('Restoring model weights from the end of the best epoch.')
            self.model.set_weights(self.best_weights)     

# Custom Loss

def quantileLoss(y_true, y_pred):
    quantiles = tf.constant([0.4, 0.5, 0.6])
    e = y_true - y_pred
    v = tf.maximum(quantiles * e, (quantiles-1) * e)
    return tf.reduce_mean(v)


class ConvImageSequenceFeatures(models.Model):
    def __init__(self, reg_factor=1e-4):
        super(ConvImageSequenceFeatures, self).__init__()
        
        self.block0 = IdentityBlock(filters=[16, 16, 16], kernel_size=(3, 3), reg_factor=reg_factor)
        self.block1 = IdentityBlock(filters=[16, 16, 16], kernel_size=(3, 3), reg_factor=reg_factor)
        self.block2 = IdentityBlock(filters=[32, 32, 32], kernel_size=(3, 3), reg_factor=reg_factor)
        self.block3 = IdentityBlock(filters=[32, 32, 32], kernel_size=(3, 3), reg_factor=reg_factor)
        self.block4 = IdentityBlock(filters=[64, 64, 64], kernel_size=(3, 3), reg_factor=reg_factor)
        
        self.conv0 = layers.Conv2D(filters=8, kernel_size=(3, 3), 
                                  padding='same', kernel_regularizer=regularizers.l2(reg_factor))
        self.bn0 = layers.BatchNormalization()
        
        self.conv1 = layers.Conv2D(filters=16, kernel_size=(3, 3), 
                                  padding='same', kernel_regularizer=regularizers.l2(reg_factor))
        self.conv2 = layers.Conv2D(filters=16, kernel_size=(3, 3), 
                                  padding='same', kernel_regularizer=regularizers.l2(reg_factor))
        self.conv3 = layers.Conv2D(filters=32, kernel_size=(3, 3), 
                                  padding='same', kernel_regularizer=regularizers.l2(reg_factor))
        self.conv4 = layers.Conv2D(filters=32, kernel_size=(3, 3), 
                                    padding='same', kernel_regularizer=regularizers.l2(reg_factor))
        self.conv5 = layers.Conv2D(filters=64, kernel_size=(3, 3), 
                                  padding='same', kernel_regularizer=regularizers.l2(reg_factor))
        
        self.drop0 = layers.Dropout(0.2)
        self.drop1 = layers.Dropout(0.2)
        self.drop2 = layers.Dropout(0.2)
        self.drop3 = layers.Dropout(0.3)
        self.drop4 = layers.Dropout(0.3)
        
        self.max_pool0 = layers.MaxPool2D((1, 2))
        self.max_pool1 = layers.MaxPool2D((1, 2))
        self.max_pool2 = layers.MaxPool2D((2, 2))
        self.max_pool3 = layers.MaxPool2D((2, 2))
        self.max_pool4 = layers.MaxPool2D((2, 2))
        
    
    def call(self, inputs, training):
        
        x = self.conv0(inputs)
        x = self.bn0(x)
        x = tf.nn.relu(x)
        x = self.max_pool0(x)
        
        x = self.drop0(x, training=training)
        x = self.conv1(x)
        x = self.block0(x)
        x = self.max_pool1(x)
        
        x = self.drop1(x, training=training)
        x = self.conv2(x)
        x = self.block1(x)
        x = self.max_pool2(x)
        
        x = self.drop2(x, training=training)
        x = self.conv3(x)
        x = self.block2(x)
        x = self.max_pool3(x)
        
        x = self.drop3(x, training=training)
        x = self.conv4(x)
        x = self.block3(x)
        x = self.max_pool4(x)
        
        x = self.drop4(x, training=training)
        x = self.conv5(x)
        x = self.block4(x)

        return x
    
class IdentityBlock(models.Model):
    def __init__(self, filters=[32, 32, 32], kernel_size=(3, 3), reg_factor=1e-4):
            super(IdentityBlock, self).__init__()

            self.l0 = layers.Conv2D(filters=filters[0], kernel_size=(1, 1), 
                              padding='valid', kernel_regularizer=regularizers.l2(reg_factor))
            self.l1 = layers.BatchNormalization()

            self.l2 = layers.Conv2D(filters=filters[1], kernel_size=kernel_size, 
                              padding='same', kernel_regularizer=regularizers.l2(reg_factor))
            self.l3 = layers.BatchNormalization()

            self.l4 = layers.Conv2D(filters=filters[2], kernel_size=(1, 1), 
                              padding='valid', kernel_regularizer=regularizers.l2(reg_factor))
            self.l5 = layers.BatchNormalization()
        
    def call(self, inputs, training):
        x_residual = inputs
        
        x = self.l0(inputs)
        x = self.l1(x)
        x = tf.nn.relu(x)
        
        x = self.l2(x)
        x = self.l3(x)
        x = tf.nn.relu(x)
        
        x = self.l4(x)
        x = self.l5(x)
        
        x = tf.add(x, x_residual)
        x = tf.nn.relu(x)
        
        return x


def buildModel(reg_factor, summary=False):
    
    block0 = ConvImageSequenceFeatures(reg_factor)
    block1 = ConvImageSequenceFeatures(reg_factor)
    block2 = ConvImageSequenceFeatures(reg_factor)
    block3 = ConvImageSequenceFeatures(reg_factor)
    block4 = ConvImageSequenceFeatures(reg_factor)
    block5 = ConvImageSequenceFeatures(reg_factor)
    block6 = ConvImageSequenceFeatures(reg_factor)
    block7 = ConvImageSequenceFeatures(reg_factor)
    block8 = ConvImageSequenceFeatures(reg_factor)
    block9 = ConvImageSequenceFeatures(reg_factor)
    
    in_0 = layers.Input(shape=(128, 469, 1))
    in_1 = layers.Input(shape=(128, 469, 1))
    in_2 = layers.Input(shape=(128, 469, 1))
    in_3 = layers.Input(shape=(128, 469, 1))
    in_4 = layers.Input(shape=(128, 469, 1))
    in_5 = layers.Input(shape=(128, 469, 1))
    in_6 = layers.Input(shape=(128, 469, 1))
    in_7 = layers.Input(shape=(128, 469, 1))
    in_8 = layers.Input(shape=(128, 469, 1))
    in_9 = layers.Input(shape=(128, 469, 1))
    
    x0 = block0(in_0)
    x1 = block1(in_1)
    x2 = block2(in_2)
    x3 = block3(in_3)
    x4 = block4(in_4)
    x5 = block5(in_5)
    x6 = block6(in_6)
    x7 = block7(in_7)
    x8 = block8(in_8)
    x9 = block9(in_9)
    
    ##################################################
    
    x0 = layers.GlobalAveragePooling2D()(x0)
    x1 = layers.GlobalAveragePooling2D()(x1)
    x2 = layers.GlobalAveragePooling2D()(x2)
    x3 = layers.GlobalAveragePooling2D()(x3)
    x4 = layers.GlobalAveragePooling2D()(x4)
    x5 = layers.GlobalAveragePooling2D()(x5)
    x6 = layers.GlobalAveragePooling2D()(x6)
    x7 = layers.GlobalAveragePooling2D()(x7)
    x8 = layers.GlobalAveragePooling2D()(x8)
    x9 = layers.GlobalAveragePooling2D()(x9)
    
    x = layers.concatenate([x0, x1, x2, x3, x4, x5, x6, x7, x8, x9])
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    
    ##################################################
    
    out_1 = layers.Dense(1, activation='relu', name='time_to_eruption')(x)
    out_2 = layers.Dense(3, activation='relu', name='quantile')(x)
    
    model = models.Model(inputs=[in_0, in_1, in_2, in_3, in_4,
                                 in_5, in_6, in_7, in_8, in_9], 
                         outputs=[out_1, out_2])

    model.compile(optimizer=optimizers.Adam(learning_rate=8e-4), 
                  
                  loss=[losses.MeanAbsoluteError(), quantileLoss],
                  loss_weights=[4, 1],
                  metrics=['mae'])
    if summary:
        print(model.summary())
    
    return model


def scheduler(epoch, lr):
    if epoch % 10 == 0:
        return lr*0.9
    else:
        return lr

    
#############################################################

In [7]:
#############################################################
# 7. Training

list_segments_train = list(unique_segments_id_train)

batch_size = 8

callback_early_stopping = ReturnBestEarlyStopping(monitor='val_time_to_eruption_loss', patience=20, 
                                                  verbose=1, restore_best_weights=True)
callback_lrsched = tf.keras.callbacks.LearningRateScheduler(scheduler)

list_callbacks = [callback_early_stopping, callback_lrsched]

kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=12)
list_history, list_models = [], []

for num_fold, (train_index, val_index) in enumerate(kf.split(list_segments_train,
                                                             np.zeros(len(list_segments_train)))):
    segments_train_fold = np.asarray(list_segments_train)[train_index]
    segments_val_fold = np.asarray(list_segments_train)[val_index]

    X_train_generator = VolcanoSequencesGenerator(segments_train_fold, dict_segments_spectograms_paths_train,
                                                  batch_size=batch_size, dict_labels=dict_labels, 
                                                  transforms=getTrainTransforms(),
                                                  training=True)

    X_val_generator = VolcanoSequencesGenerator(segments_val_fold, dict_segments_spectograms_paths_train,
                                                 batch_size=batch_size, dict_labels=dict_labels, 
                                                transforms=None,
                                                training=True)

    print(f'Num Fold: {num_fold + 1}')
    print(f'Train segments: {len(train_index)} Val segments: {len(val_index)}')

    model = buildModel(reg_factor=1e-5, summary=True)

    history = model.fit(X_train_generator,
                        validation_data=X_val_generator,
                        batch_size=batch_size,
                        callbacks=list_callbacks,
                        epochs=100,
                        verbose=1)

    list_history.append(history)
    list_models.append(model)

    X_val_batch, y_val_target = X_val_generator.generateOrderedSequences(list(segments_val_fold))
    y_pred_val = model.predict(X_val_batch, verbose=1)
    y_pred_val[0], y_pred_val[1] = y_pred_val[0]*(10**6), y_pred_val[1]*(10**6)
    y_val_target = y_val_target*(10**6)
    mae = np.abs(y_val_target - y_pred_val[0])

    model.save(f'./models/model_cnn2d_STFT_{num_fold}')

    print('***'*20)
    print(f'Prediction MAE: {mae.mean()}')
    print('***'*20)

#############################################################

Num Fold: 1
Train segments: 3544 Val segments: 887
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 469, 1) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128, 469, 1) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 128, 469, 1) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128, 469, 1) 0                                            
____________________________________

Epoch 1/100
443/443 [==============================] - 393s 887ms/step - loss: 53.3729 - time_to_eruption_loss: 11.5774 - quantile_loss: 6.9568 - time_to_eruption_mae: 11.5774 - quantile_mae: 13.7617 - val_loss: 55.3766 - val_time_to_eruption_loss: 12.2961 - val_quantile_loss: 6.0846 - val_time_to_eruption_mae: 12.2961 - val_quantile_mae: 12.4462
Epoch 2/100
443/443 [==============================] - 114s 256ms/step - loss: 45.9926 - time_to_eruption_loss: 10.1954 - quantile_loss: 5.1019 - time_to_eruption_mae: 10.1954 - quantile_mae: 10.6110 - val_loss: 46.1730 - val_time_to_eruption_loss: 10.2307 - val_quantile_loss: 5.1400 - val_time_to_eruption_mae: 10.2307 - val_quantile_mae: 10.6929
Epoch 3/100
443/443 [==============================] - 113s 255ms/step - loss: 43.7612 - time_to_eruption_loss: 9.6927 - quantile_loss: 4.8788 - time_to_eruption_mae: 9.6927 - quantile_mae: 10.1559 - val_loss: 39.0355 - val_time_to_eruption_loss: 8.6675 - val_quantile_loss: 4.2527 - val_time_to_erupti

Epoch 25/100
443/443 [==============================] - 114s 256ms/step - loss: 25.4932 - time_to_eruption_loss: 5.6396 - quantile_loss: 2.7668 - time_to_eruption_mae: 5.6396 - quantile_mae: 5.7415 - val_loss: 25.5297 - val_time_to_eruption_loss: 5.6413 - val_quantile_loss: 2.7957 - val_time_to_eruption_mae: 5.6413 - val_quantile_mae: 5.7796
Epoch 26/100
443/443 [==============================] - 113s 255ms/step - loss: 25.2618 - time_to_eruption_loss: 5.5868 - quantile_loss: 2.7447 - time_to_eruption_mae: 5.5868 - quantile_mae: 5.6919 - val_loss: 22.1058 - val_time_to_eruption_loss: 4.8884 - val_quantile_loss: 2.3810 - val_time_to_eruption_mae: 4.8884 - val_quantile_mae: 4.9562
Epoch 27/100
443/443 [==============================] - 113s 255ms/step - loss: 25.3177 - time_to_eruption_loss: 5.5991 - quantile_loss: 2.7492 - time_to_eruption_mae: 5.5991 - quantile_mae: 5.7109 - val_loss: 28.3138 - val_time_to_eruption_loss: 6.2619 - val_quantile_loss: 3.0930 - val_time_to_eruption_mae: 6.

Epoch 49/100
443/443 [==============================] - 113s 255ms/step - loss: 20.2524 - time_to_eruption_loss: 4.4621 - quantile_loss: 2.1902 - time_to_eruption_mae: 4.4621 - quantile_mae: 4.5518 - val_loss: 19.2938 - val_time_to_eruption_loss: 4.2453 - val_quantile_loss: 2.0978 - val_time_to_eruption_mae: 4.2453 - val_quantile_mae: 4.3839
Epoch 50/100
443/443 [==============================] - 113s 255ms/step - loss: 20.7273 - time_to_eruption_loss: 4.5677 - quantile_loss: 2.2413 - time_to_eruption_mae: 4.5677 - quantile_mae: 4.6515 - val_loss: 19.0593 - val_time_to_eruption_loss: 4.1998 - val_quantile_loss: 2.0439 - val_time_to_eruption_mae: 4.1998 - val_quantile_mae: 4.2618
Epoch 51/100
443/443 [==============================] - 113s 255ms/step - loss: 20.0984 - time_to_eruption_loss: 4.4272 - quantile_loss: 2.1730 - time_to_eruption_mae: 4.4272 - quantile_mae: 4.5208 - val_loss: 19.3760 - val_time_to_eruption_loss: 4.2659 - val_quantile_loss: 2.0951 - val_time_to_eruption_mae: 4.

Epoch 73/100
443/443 [==============================] - 113s 255ms/step - loss: 17.3450 - time_to_eruption_loss: 3.8089 - quantile_loss: 1.8674 - time_to_eruption_mae: 3.8089 - quantile_mae: 3.8797 - val_loss: 16.9107 - val_time_to_eruption_loss: 3.7095 - val_quantile_loss: 1.8302 - val_time_to_eruption_mae: 3.7095 - val_quantile_mae: 3.7756
Epoch 74/100
443/443 [==============================] - 113s 254ms/step - loss: 17.8272 - time_to_eruption_loss: 3.9162 - quantile_loss: 1.9194 - time_to_eruption_mae: 3.9162 - quantile_mae: 3.9844 - val_loss: 15.0260 - val_time_to_eruption_loss: 3.2921 - val_quantile_loss: 1.6142 - val_time_to_eruption_mae: 3.2921 - val_quantile_mae: 3.3627
Epoch 75/100
443/443 [==============================] - 113s 255ms/step - loss: 17.3664 - time_to_eruption_loss: 3.8132 - quantile_loss: 1.8696 - time_to_eruption_mae: 3.8132 - quantile_mae: 3.8833 - val_loss: 17.9213 - val_time_to_eruption_loss: 3.9369 - val_quantile_loss: 1.9292 - val_time_to_eruption_mae: 3.

...

Epoch 1/100
444/444 [==============================] - 202s 455ms/step - loss: 51.9462 - time_to_eruption_loss: 11.4208 - quantile_loss: 6.1561 - time_to_eruption_mae: 11.4208 - quantile_mae: 12.5253 - val_loss: 55.1530 - val_time_to_eruption_loss: 12.2387 - val_quantile_loss: 6.0899 - val_time_to_eruption_mae: 12.2387 - val_quantile_mae: 12.5682
Epoch 2/100
444/444 [==============================] - 173s 389ms/step - loss: 44.9199 - time_to_eruption_loss: 9.9547 - quantile_loss: 4.9917 - time_to_eruption_mae: 9.9547 - quantile_mae: 10.4034 - val_loss: 45.2452 - val_time_to_eruption_loss: 10.0398 - val_quantile_loss: 4.9752 - val_time_to_eruption_mae: 10.0398 - val_quantile_mae: 10.4444
Epoch 3/100
444/444 [==============================] - 179s 403ms/step - loss: 42.8934 - time_to_eruption_loss: 9.5048 - quantile_loss: 4.7621 - time_to_eruption_mae: 9.5048 - quantile_mae: 9.9063 - val_loss: 41.7547 - val_time_to_eruption_loss: 9.2444 - val_quantile_loss: 4.6644 - val_time_to_eruption_

Epoch 25/100
444/444 [==============================] - 160s 361ms/step - loss: 27.6194 - time_to_eruption_loss: 6.1168 - quantile_loss: 2.9886 - time_to_eruption_mae: 6.1168 - quantile_mae: 6.2201 - val_loss: 26.1660 - val_time_to_eruption_loss: 5.7904 - val_quantile_loss: 2.8397 - val_time_to_eruption_mae: 5.7904 - val_quantile_mae: 5.9466
Epoch 26/100
444/444 [==============================] - 160s 360ms/step - loss: 26.5040 - time_to_eruption_loss: 5.8639 - quantile_loss: 2.8826 - time_to_eruption_mae: 5.8639 - quantile_mae: 5.9936 - val_loss: 25.7353 - val_time_to_eruption_loss: 5.6894 - val_quantile_loss: 2.8107 - val_time_to_eruption_mae: 5.6894 - val_quantile_mae: 5.8338
Epoch 27/100
444/444 [==============================] - 161s 362ms/step - loss: 26.5295 - time_to_eruption_loss: 5.8698 - quantile_loss: 2.8820 - time_to_eruption_mae: 5.8698 - quantile_mae: 5.9883 - val_loss: 21.9339 - val_time_to_eruption_loss: 4.8473 - val_quantile_loss: 2.3748 - val_time_to_eruption_mae: 4.

Epoch 49/100
444/444 [==============================] - 195s 438ms/step - loss: 21.3988 - time_to_eruption_loss: 4.7175 - quantile_loss: 2.3141 - time_to_eruption_mae: 4.7175 - quantile_mae: 4.8012 - val_loss: 16.2198 - val_time_to_eruption_loss: 3.5633 - val_quantile_loss: 1.7510 - val_time_to_eruption_mae: 3.5633 - val_quantile_mae: 3.6815
Epoch 50/100
444/444 [==============================] - 193s 435ms/step - loss: 21.1117 - time_to_eruption_loss: 4.6538 - quantile_loss: 2.2796 - time_to_eruption_mae: 4.6538 - quantile_mae: 4.7433 - val_loss: 22.0088 - val_time_to_eruption_loss: 4.8501 - val_quantile_loss: 2.3904 - val_time_to_eruption_mae: 4.8501 - val_quantile_mae: 4.9970
Epoch 51/100
444/444 [==============================] - 194s 437ms/step - loss: 20.4933 - time_to_eruption_loss: 4.5150 - quantile_loss: 2.2147 - time_to_eruption_mae: 4.5150 - quantile_mae: 4.5984 - val_loss: 22.2230 - val_time_to_eruption_loss: 4.9021 - val_quantile_loss: 2.3953 - val_time_to_eruption_mae: 4.

Epoch 73/100
444/444 [==============================] - 199s 449ms/step - loss: 18.2252 - time_to_eruption_loss: 4.0034 - quantile_loss: 1.9638 - time_to_eruption_mae: 4.0034 - quantile_mae: 4.0729 - val_loss: 15.7062 - val_time_to_eruption_loss: 3.4405 - val_quantile_loss: 1.6956 - val_time_to_eruption_mae: 3.4405 - val_quantile_mae: 3.5719
Epoch 74/100
444/444 [==============================] - 195s 439ms/step - loss: 17.9598 - time_to_eruption_loss: 3.9442 - quantile_loss: 1.9342 - time_to_eruption_mae: 3.9442 - quantile_mae: 4.0174 - val_loss: 15.2367 - val_time_to_eruption_loss: 3.3362 - val_quantile_loss: 1.6426 - val_time_to_eruption_mae: 3.3362 - val_quantile_mae: 3.4500
Epoch 75/100
444/444 [==============================] - 195s 440ms/step - loss: 17.7266 - time_to_eruption_loss: 3.8912 - quantile_loss: 1.9122 - time_to_eruption_mae: 3.8912 - quantile_mae: 3.9707 - val_loss: 17.5674 - val_time_to_eruption_loss: 3.8506 - val_quantile_loss: 1.9148 - val_time_to_eruption_mae: 3.

Epoch 97/100
444/444 [==============================] - 196s 440ms/step - loss: 16.0967 - time_to_eruption_loss: 3.5245 - quantile_loss: 1.7307 - time_to_eruption_mae: 3.5245 - quantile_mae: 3.5921 - val_loss: 13.7782 - val_time_to_eruption_loss: 3.0067 - val_quantile_loss: 1.4832 - val_time_to_eruption_mae: 3.0067 - val_quantile_mae: 3.1044
Epoch 98/100
444/444 [==============================] - 196s 442ms/step - loss: 15.9567 - time_to_eruption_loss: 3.4935 - quantile_loss: 1.7142 - time_to_eruption_mae: 3.4935 - quantile_mae: 3.5637 - val_loss: 13.6643 - val_time_to_eruption_loss: 2.9832 - val_quantile_loss: 1.4626 - val_time_to_eruption_mae: 2.9832 - val_quantile_mae: 3.0510
Epoch 99/100
444/444 [==============================] - 196s 440ms/step - loss: 15.7473 - time_to_eruption_loss: 3.4467 - quantile_loss: 1.6915 - time_to_eruption_mae: 3.4467 - quantile_mae: 3.5115 - val_loss: 12.6274 - val_time_to_eruption_loss: 2.7498 - val_quantile_loss: 1.3585 - val_time_to_eruption_mae: 2.

In [10]:
#############################################################
# 8. Cross Val Score

list_segments_train = list(unique_segments_id_train)
batch_size = 8

kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=12)
df_val_all = pd.DataFrame()

for num_fold, (train_index, val_index) in tqdm(enumerate(kf.split(list_segments_train,
                                                             np.zeros(len(list_segments_train)))), 
                                               total=5, position=0):
    
    
    segments_train_fold = np.asarray(list_segments_train)[train_index]
    segments_val_fold = np.asarray(list_segments_train)[val_index]

    X_val_generator = VolcanoSequencesGenerator(segments_val_fold, dict_segments_spectograms_paths_train,
                                                batch_size=batch_size, dict_labels=dict_labels, 
                                                transforms=None,
                                                training=True)   

    model = models.load_model(f'./models/model_cnn2d_STFT_{num_fold}', compile=False)

    X_val_sequences, y_val_target = X_val_generator.generateOrderedSequences(segments_val_fold)
    y_pred_val = model.predict(X_val_sequences)

    df_tmp = pd.DataFrame({
            'pred' :  y_pred_val[0].squeeze()*(10**6),
            'pred_q_30' : y_pred_val[1][:, 0]*(10**6),
            'pred_q_50' : y_pred_val[1][:, 1]*(10**6),
            'pred_q_70' : y_pred_val[1][:, 2]*(10**6),
            'y_true' : y_val_target.flatten()*(10**6)
    })

    df_val_all = pd.concat([df_val_all, df_tmp], axis=0)

print('***'*20)
print(np.mean(np.abs(df_tmp['y_true'] - df_tmp['pred'])))
print(np.mean(np.abs(df_tmp['y_true'] - df_tmp['pred_q_50'])))
print('***'*20)

#############################################################


************************************************************
2749811.572227194
2749516.9172712113
************************************************************


In [8]:
#############################################################
# 9. Inference

batch_size_prediction = 8

# del X_val_sequences, y_val_target, list_cv_pred, y_pred_cv, y_cv_target, df_cv, X_cv_sequences, y_cv_target
gc.collect()

list_models = [models.load_model(f'./models/model_cnn2d_STFT_{i}', compile=False) for i in range(5)]

X_test_generator = VolcanoSequencesGenerator(list(unique_segments_id_test), dict_segments_spectograms_paths_test,
                                             batch_size=batch_size_prediction, dict_labels=dict_labels, 
                                             transforms=None,
                                             training=False)


idx = 0
num_test_steps = int(np.ceil(len(unique_segments_id_test) / batch_size_prediction))
list_test_segments = list(unique_segments_id_test)
array_predictions = np.zeros((len(list_test_segments)))

for i in tqdm(range(num_test_steps), total=num_test_steps, position=0):
    list_tmp_segments = list_test_segments[idx:(idx+batch_size_prediction)]
    X_test_sequences = X_test_generator.generateOrderedSequences(list_tmp_segments)
     
    predictions = [model.predict(X_test_sequences)[0].squeeze() for model in list_models]
    predictions = np.mean(np.asarray(predictions), axis=0)*(10**6)
    array_predictions[idx:(idx+batch_size_prediction)] = predictions
    idx += batch_size_prediction   
    del predictions
    gc.collect()

df_submission = pd.DataFrame({
    'segment_id' : list_test_segments,
    'time_to_eruption' : array_predictions
})

df_submission.to_csv('./FinalSubmissions/' + 'submission_cnn2d_stft.csv', index=False)
df_submission.describe()

#############################################################

,segment_id,time_to_eruption
count,4.520000e+03,4.520000e+03
mean,1.066993e+09,2.392074e+07
std,6.162904e+08,1.079330e+07
min,8.602880e+05,1.046912e+06
25%,5.458995e+08,1.744937e+07
50%,1.060695e+09,2.476303e+07
75%,1.599284e+09,3.197190e+07
max,2.147116e+09,4.889708e+07
